Send Feedback
Titanic dataset is one of the datasets available in sklearn.
You are given:
    1. A Training dataset csv file with X train and Y train data
    2. A X test File and you have to predict and submit predictions for this file.
Your task is to:
    1. Use Logistic Regression and come with predictions.
Read Instructions carefully -
    1. Use Logistic Regression as a training algorithm and submit results predicted.
    2. Files are in csv format.
    3. Submit a csv file with only predictions for X test data. File should not have any headers and should only have one column i.e. predictions.
    4. Your score is based on number of accurate predictions.

In [2]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA


In [33]:
# Loading Dataset
dataframe = pd.read_csv("training_titanic_x_y_train.csv", delimiter = ",")
dataframe.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.000,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.050,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.000,NaN,S,0
3,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,0
4,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,0


In [34]:
dataframe.describe()

,Pclass,Age,SibSp,Parch,Fare,Survived
count,668.000000,536.000000,668.000000,668.000000,668.000000,668.000000
mean,2.296407,29.700560,0.528443,0.407186,32.064552,0.402695
std,0.831638,14.240257,1.080327,0.854695,45.320835,0.490808
min,1.000000,0.670000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.925000,0.000000
50%,3.000000,29.000000,0.000000,0.000000,14.750000,0.000000
75%,3.000000,38.250000,1.000000,0.000000,31.275000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [35]:
df = dataframe.copy()
sex = df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(0 if s == 'male' else 1)
df['Sex'] = sex_int
df.drop('Name', axis = 1, inplace = True)
df.drop('Ticket', axis = 1, inplace = True)
df.drop('Cabin', axis = 1, inplace = True)

embarked_counts = { i :(df['Embarked'] == i).sum() for i in set(df['Embarked'])}

def embarked_int(s):
    return embarked_counts[s]
df['Embarked'] = [embarked_int(i) for i in df['Embarked']]
#df.drop('Embarked', axis = 1, inplace = True)

ages = df['Age'].fillna(0).values
ages_nonzero = ages[np.nonzero(ages)]

mean_age = np.array(ages_nonzero).mean()


survived_female_ages = np.array(df['Survived'] * (df['Sex']) * ages )
indices_sfa = np.nonzero(survived_female_ages)
count_sfa = len(survived_female_ages[indices_sfa])
sum_sfa = np.sum(survived_female_ages)
mean_sfa = sum_sfa/count_sfa

not_survived_female_ages = np.array((1 - df['Survived']) * (df['Sex']) * ages )
indices_nsfa = np.nonzero(not_survived_female_ages)
count_nsfa = len(not_survived_female_ages[indices_nsfa])
sum_nsfa = np.sum(not_survived_female_ages)
mean_nsfa = sum_nsfa/count_nsfa

survived_male_ages = np.array(df['Survived'] * (1-df['Sex']) * ages )
indices_sma = np.nonzero(survived_male_ages)

count_sma = len(survived_male_ages[indices_sma])
sum_sma = np.sum(survived_male_ages)
mean_sma = sum_sma/count_sma

not_survived_male_ages = np.array((1 - df['Survived']) * (1-df['Sex']) * ages )
indices_nsma = np.nonzero(not_survived_male_ages)

count_nsma = len(not_survived_male_ages[indices_nsma])
sum_nsma = np.sum(not_survived_male_ages)
mean_nsma = sum_nsma/count_nsma

print(mean_sma, mean_sfa, mean_nsma, mean_nsfa)
df.describe()

27.4375 28.4884868421 31.7065637066 26.3673469388


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
count,668.000000,668.000000,536.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,2.296407,0.360778,29.700560,0.528443,0.407186,32.064552,380.905689,0.402695
std,0.831638,0.480586,14.240257,1.080327,0.854695,45.320835,168.501612,0.490808
min,1.000000,0.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.925000,133.000000,0.000000
50%,3.000000,0.000000,29.000000,0.000000,0.000000,14.750000,484.000000,0.000000
75%,3.000000,1.000000,38.250000,1.000000,0.000000,31.275000,484.000000,1.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,484.000000,1.000000


In [37]:
#print(training_final_df.shape)
#print(df.shape)
values = np.empty((0,8))
for row in df.values :
    #print(row[2])
    if np.isnan(row[2]) : 
        if row[1] == 0.0 : # male
            if row[-1] == 1.0: # survived 
                row[2] = mean_sma
            else:
                row[2] = mean_nsma
        else: # female
            if row[-1] == 1.0: # survived
                row[2] = mean_sfa
            else:
                row[2] = mean_nsfa
    values = np.insert(values,0, row, axis = 0) 
    #training_final_df.append(row)
training_final_df = pd.DataFrame(values, columns = df.columns)

print(training_final_df.describe())
#print(values)


           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  668.000000  668.000000  668.000000  668.000000  668.000000  668.000000   
mean     2.296407    0.360778   29.783072    0.528443    0.407186   32.064552   
std      0.831638    0.480586   12.786115    1.080327    0.854695   45.320835   
min      1.000000    0.000000    0.670000    0.000000    0.000000    0.000000   
25%      2.000000    0.000000   23.000000    0.000000    0.000000    7.925000   
50%      3.000000    0.000000   29.000000    0.000000    0.000000   14.750000   
75%      3.000000    1.000000   35.000000    1.000000    0.000000   31.275000   
max      3.000000    1.000000   80.000000    8.000000    6.000000  512.329200   

         Embarked    Survived  
count  668.000000  668.000000  
mean   380.905689    0.402695  
std    168.501612    0.490808  
min      0.000000    0.000000  
25%    133.000000    0.000000  
50%    484.000000    0.000000  
75%    484.000000    1.000000  
max    484.000

In [38]:
final_X_train = training_final_df.values
print(final_X_train.shape)
X_train = training_final_df.drop('Survived', axis = 1).values
Y_train = training_final_df['Survived'].values
print(X_train.shape)
print(Y_train.shape)


(668, 8)
(668, 7)
(668,)


In [39]:
df.fillna(mean_age, inplace = True)
#print(df)
#df.describe()

X_df = df.drop('Survived', axis = 1)
X_train = X_df.values
Y_train = df['Survived'].values

print("Shape of X_train ", X_train.shape)
print("Shape of Y_train ", Y_train.shape)

test_dataframe = pd.read_csv("test_titanic_x_test.csv", delimiter = ",")


test_df = test_dataframe.copy()
sex = test_df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(0 if s == 'male' else 1)
test_df['Sex'] = sex_int
test_df.drop('Name', axis = 1, inplace = True)
test_df.drop('Ticket', axis = 1, inplace = True)
test_df.drop('Cabin', axis = 1, inplace = True)

test_df['Embarked'] = [embarked_int(i) for i in test_df['Embarked']]

#df.drop('Embarked', axis = 1, inplace = True)

ages = test_df['Age'].fillna(0).values
ages_nonzero = ages[np.nonzero(ages)]

X_test = test_df.values

print("Shape of X_test ", X_test.shape)


Shape of X_train  (668, 7)
Shape of Y_train  (668,)
Shape of X_test  (223, 7)


In [40]:
mean_fa = (mean_sfa * count_sfa + mean_nsfa * count_nsfa)/(count_nsfa + count_sfa)
mean_ma = (mean_sma * count_sma + mean_nsma * count_nsma)/(count_nsma + count_sma)

values = np.empty((0,7))
for row in test_df.values :
    #print(row[2])
    if np.isnan(row[2]) : 
        if row[1] == 1.0 : # male
            row[2] = mean_ma
        else: # female
            row[2] = mean_fa
    values = np.insert(values,0, row, axis = 0) 
    #training_final_df.append(row)
testing_final_df = pd.DataFrame(values, columns = test_df.columns)

print(testing_final_df.describe())
#print(values)


           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  223.000000  223.000000  223.000000  223.000000  223.000000  223.000000   
mean     2.345291    0.327354   29.508293    0.506726    0.304933   32.622551   
std      0.850047    0.470303   13.765760    1.169700    0.634108   61.062047   
min      1.000000    0.000000    0.420000    0.000000    0.000000    0.000000   
25%      2.000000    0.000000   22.000000    0.000000    0.000000    7.879200   
50%      3.000000    0.000000   27.971393    0.000000    0.000000   12.475000   
75%      3.000000    1.000000   35.000000    1.000000    0.000000   30.035400   
max      3.000000    1.000000   71.000000    8.000000    2.000000  512.329200   

         Embarked  
count  223.000000  
mean   374.192825  
std    176.847452  
min      0.000000  
25%    133.000000  
50%    484.000000  
75%    484.000000  
max    484.000000  


In [41]:
X_test = testing_final_df.values
print(X_test.shape)

(223, 7)


In [42]:
import sklearn.preprocessing as pps

def add_more_features (X_train, imp_cols_indices = []):
    
    X_train_df = pd.DataFrame(X_train)
    num_f = len(X_train_df.columns)
    col_names = X_train_df.columns
    
    if len(imp_cols_indices) == 0 :
        imp_cols_indices = np.arange(num_f)
    new_df = X_train_df.copy()
    num_imp_cols = len(imp_cols_indices)
    
    pow = 3
    
    while pow < 3 :
        for i1 in range(num_imp_cols) :
            i = imp_cols_indices[i1]
            new_df[ str(col_names[i]) + "_pow_" + str(pow) ] = X_train_df[i] ** pow
        pow += 1
    
    for i1 in range(num_imp_cols) :
        for j1 in range(i1, num_imp_cols):
            i = imp_cols_indices[i1]
            j = imp_cols_indices[j1]
            new_df[ str(col_names[i]) + "_" + str(col_names[j])] = X_train_df[i] * X_train_df[j]
            #new_df[ str(col_names[i])*2 + "_" + str(col_names[j])] = X_train_df[i]**2 * X_train_df[j]
            #new_df[ str(col_names[i]) + "_" + str(col_names[j])*2 ] = X_train_df[i] * X_train_df[j]**2
            #new_df[ str(col_names[i])*2 + "_" + str(col_names[j])*2 ] = X_train_df[i]**2 * X_train_df[j]**2
            
            #new_df[ str(col_names[i])*3 + "_" + str(col_names[j])] = X_train_df[i]**3 * X_train_df[j]
            #new_df[ str(col_names[i])*3 + "_" + str(col_names[j])*2] = X_train_df[i]**3 * X_train_df[j]**2
            #new_df[ str(col_names[i])*3 + "_" + str(col_names[j])*3 ] = X_train_df[i]**3 * X_train_df[j]**3
            #new_df[ str(col_names[i])*2 + "_" + str(col_names[j])*3 ] = X_train_df[i]**2 * X_train_df[j]**3
            #new_df[ str(col_names[i]) + "_" + str(col_names[j])*3 ] = X_train_df[i] * X_train_df[j]**3
              
    print(new_df.describe())
    return new_df.values


X_train_new = add_more_features(X_train)
X_test_new = add_more_features(X_test)

std_scaler = pps.StandardScaler()
std_scaler.fit(X_train_new)

X_train_scaled = std_scaler.transform(X_train_new)
X_test_scaled = std_scaler.transform(X_test_new)

import copy
X_train_original = copy.deepcopy(X_train)
X_test_original = copy.deepcopy(X_test)

#X_train = X_train_scaled
#X_test = X_test_scaled

print(X_train.shape)
print(X_test.shape)


                0           1           2           3           4           5  \
count  668.000000  668.000000  668.000000  668.000000  668.000000  668.000000   
mean     2.296407    0.360778   29.700560    0.528443    0.407186   32.064552   
std      0.831638    0.480586   12.753571    1.080327    0.854695   45.320835   
min      1.000000    0.000000    0.670000    0.000000    0.000000    0.000000   
25%      2.000000    0.000000   23.000000    0.000000    0.000000    7.925000   
50%      3.000000    0.000000   29.700560    0.000000    0.000000   14.750000   
75%      3.000000    1.000000   35.000000    1.000000    0.000000   31.275000   
max      3.000000    1.000000   80.000000    8.000000    6.000000  512.329200   

                6         0_0         0_1         0_2      ...        \
count  668.000000  668.000000  668.000000  668.000000      ...         
mean   380.905689    5.964072    0.776946   65.167695      ...         
std    168.501612    3.429989    1.154981   33.863689 

In [43]:
X_t_df = pd.DataFrame(X_train)
X_t_df.describe()

,0,1,2,3,4,5,6
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,2.296407,0.360778,29.700560,0.528443,0.407186,32.064552,380.905689
std,0.831638,0.480586,12.753571,1.080327,0.854695,45.320835,168.501612
min,1.000000,0.000000,0.670000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,23.000000,0.000000,0.000000,7.925000,133.000000
50%,3.000000,0.000000,29.700560,0.000000,0.000000,14.750000,484.000000
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.275000,484.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,484.000000


In [44]:
# extract best features

from sklearn.decomposition import PCA

pca = PCA()
pca.fit(np.concatenate((X_train, X_test), axis = 0))

#X_train = pca.transform(X_train)
#X_test = pca.transform(X_test)

print(X_train.shape)
print(X_test.shape)


(668, 7)
(223, 7)


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(200, 200, 200, 200, 200))
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

print("Training Score : ", model.score(X_train, Y_train))

Training Score :  0.579341317365


In [46]:
print(Y_pred)

[1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0
 0 1 0 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 0 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1
 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 1 0
 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1
 1]


In [47]:
np.savetxt("y_pred.csv", Y_pred, fmt = "%d") 